In [18]:
import re
import math
import warnings
import tqdm
import matplotlib
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from matplotlib.pyplot import MultipleLocator
from lightgbm import LGBMClassifier
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, Binarizer
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
warnings.filterwarnings("ignore")


%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False 

In [19]:
# ! conda install tqdm 
# ! conda install pandas 
# ! conda install numpy 
# ! conda install scipy 
# ! conda install matplotlib 
# ! conda install xgboost 
# ! conda install -c conda-forge sklearn-contrib-lightning
# ! conda install -c conda-forge tpot-imblearn
# ! conda install lightgbm
# ! conda install openpyxl

In [20]:
data1=pd.read_excel('/root/2023-NPMCM-main/表1-患者列表及临床信息.xlsx')
data2=pd.read_excel('/root/2023-NPMCM-main/表2-患者影像信息血肿及水肿的体积及位置.xlsx')
data3=pd.read_excel('/root/2023-NPMCM-main/表3-患者影像信息血肿及水肿的形状及灰度分布.xlsx')
data4=pd.read_excel('/root/2023-NPMCM-main/表4-答案文件.xlsx')
data5=pd.read_excel('1b数据.xlsx').drop(['ID'], axis=1)

data1.rename(columns={data1.columns[0]: "ID"}, inplace=True)
data2.rename(columns={data2.columns[0]: "ID"}, inplace=True)

data_f_1=pd.read_excel('/root/2023-NPMCM-main/附表1-检索表格-流水号vs时间.xlsx')
data_kz=pd.read_excel('/root/2023-NPMCM-main/q11/扩张及时间.xlsx')

In [21]:
# 转换独热编码
xueya_list = []
for i in data5['血压']:
    y1 = i.split('/')[0]
    y2 = i.split('/')[1]
    if int(y1) > 139 or int(y1) < 90:
        status_i = 0
    elif int(y2) > 89 or int(y2) < 60:
        status_i = 0
    else:
        status_i = 1
    xueya_list.append(status_i)
data5['血压'] = np.array(xueya_list)
data5['性别_男'] = data5['性别'].apply(lambda x: 1 if x == '男' else 0)
data5['性别_女'] = data5['性别'].apply(lambda x: 1 if x == '女' else 0)
data5 = data5.drop(columns=['性别'])
x = data5['年龄'].values
scaler = MinMaxScaler()
x_normalized = scaler.fit_transform(x.reshape(-1, 1))
binarizer = Binarizer(threshold=0.5)
x_binarized = binarizer.fit_transform(x_normalized)
data5['年龄'] = x_binarized
data5 = data5.drop(['流水号'], axis = 1)
data5.head(100)

,年龄,脑出血前mRS评分,高血压病史,卒中病史,糖尿病史,房颤史,冠心病史,吸烟史,饮酒史,发病到首次影像检查时间间隔,...,NCCT_original_firstorder_Median.Hemo,NCCT_original_firstorder_Minimum.Hemo,NCCT_original_firstorder_Range.Hemo,NCCT_original_firstorder_RobustMeanAbsoluteDeviation.Hemo,NCCT_original_firstorder_RootMeanSquared.Hemo,NCCT_original_firstorder_Skewness.Hemo,NCCT_original_firstorder_Uniformity.Hemo,NCCT_original_firstorder_Variance.Hemo,性别_男,性别_女
0,0.0,0,0,0,0,0,0,0,0,2.5,...,180.997857,86.200064,161.239687,22.759625,175.551025,-0.523096,0.089596,1225.449943,0,1
1,0.0,0,1,0,0,0,0,0,0,3.0,...,137.569328,69.446385,152.848311,26.758880,145.014063,0.113439,0.074774,1394.452289,1,0
2,1.0,0,1,0,0,0,0,0,0,2.0,...,133.217856,61.683739,164.933560,19.037080,135.747065,0.019111,0.098694,812.571273,1,0
3,1.0,2,1,1,0,0,0,0,0,1.0,...,124.293778,59.981668,230.164456,21.618937,130.690592,0.323449,0.123988,1061.812328,1,0
4,0.0,0,0,0,0,0,0,0,0,5.0,...,146.017474,81.014412,134.699827,20.821637,148.459109,0.027286,0.075332,941.159277,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,0,0,0,0,0,0,0,0,1.0,...,142.643719,78.215613,146.557579,22.871665,145.937323,0.008035,0.080505,1056.837094,0,1
96,1.0,0,1,0,0,0,0,0,0,3.0,...,162.653881,82.822093,169.854351,29.985000,170.381262,0.062903,0.078045,1702.945541,0,1
97,1.0,0,1,1,1,1,1,1,0,2.5,...,157.553169,64.587076,157.060233,16.909283,154.139448,-0.589568,0.104148,814.067313,1,0
98,0.0,0,1,0,0,0,0,0,0,3.0,...,115.571195,71.609664,193.688206,22.412772,132.134408,0.698339,0.124429,1172.576753,1,0


In [22]:
# 归一化
# 创建 MinMaxScaler 对象
scaler = MinMaxScaler()
# 选择要归一化的列（除了 '年龄' 列之外的所有列）
columns_to_normalize = [col for col in data5.columns if col != '性别']
# 使用 MinMaxScaler 对所选列进行归一化
data5[columns_to_normalize] = scaler.fit_transform(data5[columns_to_normalize])

In [23]:
data5

,年龄,脑出血前mRS评分,高血压病史,卒中病史,糖尿病史,房颤史,冠心病史,吸烟史,饮酒史,发病到首次影像检查时间间隔,...,NCCT_original_firstorder_Median.Hemo,NCCT_original_firstorder_Minimum.Hemo,NCCT_original_firstorder_Range.Hemo,NCCT_original_firstorder_RobustMeanAbsoluteDeviation.Hemo,NCCT_original_firstorder_RootMeanSquared.Hemo,NCCT_original_firstorder_Skewness.Hemo,NCCT_original_firstorder_Uniformity.Hemo,NCCT_original_firstorder_Variance.Hemo,性别_男,性别_女
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094737,...,0.794741,0.580976,0.461355,0.596541,0.747897,0.073091,0.233654,0.575522,0.0,1.0
1,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.115789,...,0.312688,0.302489,0.415018,0.784219,0.401521,0.432276,0.062318,0.678884,1.0,0.0
2,1.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.073684,...,0.264387,0.173455,0.481753,0.421848,0.296407,0.379049,0.338820,0.323005,1.0,0.0
3,1.0,0.666667,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.031579,...,0.165330,0.145162,0.841963,0.543010,0.239052,0.550780,0.631206,0.475441,1.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,...,0.406462,0.494777,0.314800,0.505594,0.440597,0.383662,0.068760,0.401649,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,1.0,0.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.221053,...,0.606803,0.322885,0.524279,0.619519,0.577253,0.164114,0.277650,0.558400,0.0,1.0
156,0.0,0.666667,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.010526,...,0.235845,0.492805,0.332350,0.645449,0.373741,0.577202,0.118778,0.521251,1.0,0.0
157,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.107368,...,0.343452,0.357229,0.372732,0.559288,0.408873,0.436945,0.129904,0.439126,1.0,0.0
158,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.115789,...,0.196442,0.146322,0.385698,0.487636,0.229306,0.380374,0.221827,0.349817,1.0,0.0


In [24]:
train = data5[:100]
label = data_kz['是否扩张'].values[:100]
predict = data5[100:]

In [25]:
sm = SMOTE(random_state=0)
xres, yres = sm.fit_resample(train.values, label)
xtrain, xvalid, ytrain, yvalid = train_test_split(train.values,label,random_state=250,test_size=0.2)
xtrain_res, xvalid_res, ytrain_res, yvalid_res = train_test_split(xres,yres,random_state=300,test_size=0.2)

In [26]:
# 定义参数网格
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

param_grid_lgbm = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 20],
    'learning_rate': [0.01, 0.1, 0.2],
}

param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 20],
    'learning_rate': [0.01, 0.1, 0.2],
}

param_grid_svc = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
}

param_grid_mlp = {
    'hidden_layer_sizes': [(50, 50), (100, 100), (50, 100)],
    'activation': ['relu', 'tanh'],
}

param_grid_lr = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'], 
}

In [ ]:
# 采样前
def function(model, param_grid, cv_values=[5], n_jobs=16):
    best_auc = 0
    best_cv = None
    best_model = None
    best_params = None

    for cv in cv_values:
        grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='roc_auc', n_jobs=n_jobs)
        grid_search.fit(train.values, label)
        
        if grid_search.best_score_ > best_auc:
            best_model = grid_search.best_estimator_
            best_auc = grid_search.best_score_
            best_params = grid_search.best_params_
            best_cv = cv

    if best_model is None:
        raise ValueError("No model was trained successfully.")

    acc = round(cross_val_score(best_model, train.values, label, cv=best_cv, scoring='accuracy').mean(), 2)
    recall = round(cross_val_score(best_model, train.values, label, cv=best_cv, scoring='recall').mean(), 2)
    precision = round(cross_val_score(best_model, train.values, label, cv=best_cv, scoring='precision').mean(), 2)
    f1 = round(cross_val_score(best_model, train.values, label, cv=best_cv, scoring='f1').mean(), 2)
    
    result_dict = {
        'best_model': best_model,
        'best_auc': round(best_auc, 2),
        'best_params': best_params,
        'acc': acc,
        'recall': recall,
        'precision': precision,
        'f1': f1,
        'best_cv': best_cv
    }
    
    return result_dict

cv_values = [5, 10]
best_results_rf = function(RandomForestClassifier(random_state=100), param_grid_rf, cv_values)
best_results_lgbm = function(LGBMClassifier(random_state=100), param_grid_lgbm, cv_values)
best_results_xgb = function(XGBClassifier(random_state=100), param_grid_xgb, cv_values)
best_results_svc = function(SVC(probability=True, random_state=100), param_grid_svc, cv_values)
best_results_mlp = function(MLPClassifier(random_state=100), param_grid_mlp, cv_values)
best_results_lr = function(LogisticRegression(random_state=100), param_grid_lr, cv_values)

def f(model_list,name_list,types='train'):
    plt.figure(figsize=(8, 7), dpi=80, facecolor='w')    # dpi:每英寸长度的像素点数；facecolor 背景颜色
    plt.xlim((-0.01, 1.02))  # x,y 轴刻度的范围
    plt.ylim((-0.01, 1.02))
    plt.xticks(np.arange(0, 1.1, 0.1))  #绘制刻度
    plt.yticks(np.arange(0, 1.1, 0.1))
    
    if types == 'test':
        for model,name in zip(model_list,name_list):
            ytest_prob = model.predict_proba(xvalid)[:,1]
            fpr, tpr, _ = metrics.roc_curve(yvalid, ytest_prob)
            auc = metrics.auc(fpr, tpr)
            plt.plot(fpr, tpr, '-', lw=2, label=f'{name} AUC:%.5f' % auc)  # 绘制AUC 曲线
    else:
        for model,name in zip(model_list,name_list):
            ytest_prob = model.predict_proba(xtrain)[:,1]
            fpr, tpr, _ = metrics.roc_curve(ytrain, ytest_prob)
            auc = metrics.auc(fpr, tpr)
            plt.plot(fpr, tpr, '-', lw=2, label=f'{name} AUC:%.5f' % auc)  # 绘制AUC 曲线
    plt.legend(loc='upper left',fontsize=15)    # 设置显示标签的位置
    plt.xlabel('False Positive Rate', fontsize=14)   #绘制x,y 坐标轴对应的标签
    plt.ylabel('True Positive Rate', fontsize=14)
    plt.tick_params(labelsize=23)

    plt.grid(b=True, ls=':')  # 绘制网格作为底板;b是否显示网格线；ls表示line style
    plt.savefig(f'问题1\\roc_auc({types}(采样前)).png',dpi=500)
    plt.show()

model0 = RandomForestClassifier(random_state=100)
model1 = LGBMClassifier(random_state=100)
model2 = XGBClassifier(random_state=100)
model3 = SVC(probability=True,random_state=100)
model4 = MLPClassifier(random_state=100)
model5 = LogisticRegression(random_state=100)

model0.fit(xtrain,ytrain)
model1.fit(xtrain,ytrain)
model2.fit(xtrain,ytrain)
model3.fit(xtrain,ytrain)
model4.fit(xtrain,ytrain)
model5.fit(xtrain,ytrain)


In [ ]:
f([model0, model1, model2, model3, model4, model5], ['RF', 'LGBM', 'XGB', 'SVC', 'MLP', 'LR'], 'test')
f([model0, model1, model2, model3, model4, model5], ['RF', 'LGBM', 'XGB', 'SVC', 'MLP', 'LR'], 'train')

In [ ]:
# 采样后
def function(model, param_grid, cv_values=[5], n_jobs=16):
    best_auc = 0
    best_cv = None
    best_model = None
    best_params = None

    for cv in cv_values:
        grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='roc_auc', n_jobs=n_jobs)
        grid_search.fit(xres, yres)
        
        if grid_search.best_score_ > best_auc:
            best_model = grid_search.best_estimator_
            best_auc = grid_search.best_score_
            best_params = grid_search.best_params_
            best_cv = cv

    if best_model is None:
        raise ValueError("No model was trained successfully.")

    acc = round(cross_val_score(best_model, xres, yres, cv=best_cv, scoring='accuracy').mean(), 2)
    recall = round(cross_val_score(best_model, xres, yres, cv=best_cv, scoring='recall').mean(), 2)
    precision = round(cross_val_score(best_model, xres, yres, cv=best_cv, scoring='precision').mean(), 2)
    f1 = round(cross_val_score(best_model, xres, yres, cv=best_cv, scoring='f1').mean(), 2)
    
    result_dict = {
        'best_model': best_model,
        'best_auc': round(best_auc, 2),
        'best_params': best_params,
        'acc': acc,
        'recall': recall,
        'precision': precision,
        'f1': f1,
        'best_cv': best_cv
    }
    
    return result_dict

cv_values = [5, 10]
best_results_rf = function(RandomForestClassifier(random_state=100), param_grid_rf, cv_values)
best_results_lgbm = function(LGBMClassifier(random_state=100), param_grid_lgbm, cv_values)
best_results_xgb = function(XGBClassifier(random_state=100), param_grid_xgb, cv_values)
best_results_svc = function(SVC(probability=True, random_state=100), param_grid_svc, cv_values)
best_results_mlp = function(MLPClassifier(random_state=100), param_grid_mlp, cv_values)
best_results_lr = function(LogisticRegression(random_state=100), param_grid_lr, cv_values)

def f(model_list,name_list,types='train'):
    plt.figure(figsize=(8, 7), dpi=80, facecolor='w')    # dpi:每英寸长度的像素点数；facecolor 背景颜色
    plt.xlim((-0.01, 1.02))  # x,y 轴刻度的范围
    plt.ylim((-0.01, 1.02))
    plt.xticks(np.arange(0, 1.1, 0.1))  #绘制刻度
    plt.yticks(np.arange(0, 1.1, 0.1))
    
    if types == 'test':
        for model,name in zip(model_list,name_list):
            ytest_prob = model.predict_proba(xvalid_res)[:,1]
            fpr, tpr, _ = metrics.roc_curve(yvalid_res, ytest_prob)
            auc = metrics.auc(fpr, tpr)
            plt.plot(fpr, tpr, '-', lw=2, label=f'{name} AUC:%.5f' % auc)  # 绘制AUC 曲线
    else:
        for model,name in zip(model_list,name_list):
            ytest_prob = model.predict_proba(xtrain_res)[:,1]
            fpr, tpr, _ = metrics.roc_curve(ytrain_res, ytest_prob)
            auc = metrics.auc(fpr, tpr)
            plt.plot(fpr, tpr, '-', lw=2, label=f'{name} AUC:%.5f' % auc)  # 绘制AUC 曲线
    plt.legend(loc='upper left',fontsize=15)    # 设置显示标签的位置
    plt.xlabel('False Positive Rate', fontsize=14)   #绘制x,y 坐标轴对应的标签
    plt.ylabel('True Positive Rate', fontsize=14)
    plt.tick_params(labelsize=23)

    plt.grid(b=True, ls=':')  # 绘制网格作为底板;b是否显示网格线；ls表示line style
    plt.savefig(f'问题1\\roc_auc({types}（采样后）).png',dpi=500)
    plt.show()

model0 = best_results_rf['best_model']
model1 = best_results_lgbm['best_model']
model2 = best_results_xgb['best_model']
model3 = best_results_svc['best_model']
model4 = best_results_mlp['best_model']
model5 = best_results_lr['best_model']

model0.fit(xtrain_res,ytrain_res)
model1.fit(xtrain_res,ytrain_res)
model2.fit(xtrain_res,ytrain_res)
model3.fit(xtrain_res,ytrain_res)
model4.fit(xtrain_res,ytrain_res)
model5.fit(xtrain_res,ytrain_res)

In [ ]:
f([model0, model1, model2, model3, model4, model5], ['RF', 'LGBM', 'XGB', 'SVC', 'MLP', 'LR'], 'test')
f([model0, model1, model2, model3, model4, model5], ['RF', 'LGBM', 'XGB', 'SVC', 'MLP', 'LR'], 'train')

In [ ]:
xgb_model = best_results_xgb['best_model']
xgb_model.fit(xres,yres)
xgb_model_result = [round(i,4) for i in xgb_model.predict_proba(data5.values)[:,1]]

In [ ]:
xgb_model_result = pd.DataFrame({'扩张概率':xgb_model_result})
xgb_model_result.to_excel('q12扩张概率.xlsx',index=False)

In [ ]:
# 'name':data1['ID'].values,
xgb_model_result